In [1]:
import json
from random import Random
import pandas as pd

import sqlparse

In [2]:
atis_json_file = "../data/atis/atis.json"

In [3]:
with open(atis_json_file) as in_fp:
    atis_json = json.load(in_fp)

In [4]:
len(atis_json)

947

In [24]:
data = []

for index, line in enumerate(atis_json):
    fixed_sentences = []
    for sentence in line["sentences"]:
        text = sentence["text"]
        variables = sentence["variables"]
        for var_name, var_value in variables.items():
            text = text.replace(var_name, var_value)
        fixed_sentences.append(text)
    queries = line["sql"]
    variables_list = line["variables"]
    
    fixed_queries = []
    for query in queries:
        for variable_json in variables_list:
            query = query.replace(variable_json["name"], variable_json["example"])
        fixed_queries.append(query)

    data.append((fixed_sentences[0], fixed_queries[0]))

In [28]:
random = Random(42)

In [29]:
random.shuffle(data)

In [30]:
titles = [item[0] for item in data]

In [31]:
sql_queries = [item[1] for item in data]

In [33]:
data_df = pd.DataFrame(data, columns=["Title", "SQL"])

In [34]:
data_df.head()

,Title,SQL
0,how much is a FIRST class ticket from WASHINGT...,SELECT DISTINCT FAREalias0.FARE_ID FROM AIRPOR...
1,list flights from CHARLOTTE on saturday afternoon,SELECT DISTINCT FLIGHTalias0.FLIGHT_ID FROM FL...
2,how much is an economy ticket from WASHINGTON ...,SELECT DISTINCT FAREalias0.FARE_ID FROM AIRPOR...
3,i want to go from DENVER to ATLANTA round trip...,SELECT DISTINCT FAREalias0.FARE_ID FROM AIRPOR...
4,what airport in NEW YORK is closest to downtown,SELECT DISTINCT AIRPORTalias0.AIRPORT_CODE FRO...


In [43]:
data_df["SQL"] = data_df["SQL"].apply(lambda sql: sqlparse.format(sql, reindent=True, keyword_case="upper"))

In [44]:
data_df.head()

,Title,SQL
0,how much is a FIRST class ticket from WASHINGT...,SELECT DISTINCT FAREalias0.FARE_ID\nFROM AIRPO...
1,list flights from CHARLOTTE on saturday afternoon,SELECT DISTINCT FLIGHTalias0.FLIGHT_ID\nFROM F...
2,how much is an economy ticket from WASHINGTON ...,SELECT DISTINCT FAREalias0.FARE_ID\nFROM AIRPO...
3,i want to go from DENVER to ATLANTA round trip...,SELECT DISTINCT FAREalias0.FARE_ID\nFROM AIRPO...
4,what airport in NEW YORK is closest to downtown,SELECT DISTINCT AIRPORTalias0.AIRPORT_CODE\nFR...


In [45]:
with open("../data/atis/atis_analysis.csv", "w+") as out_fp:
    data_df.to_csv(out_fp, index=False, line_terminator="\n")